# Анализ  А/A/B - теста для мобильного приложения

Стартап по продаже продуктов питания запустил приложение, необходимо составить картину пользовательского поведения. Также у дизайнеров стартапа возникла идея сменить шрифты в приложении. По-мнению менджеров, данное предложение весьма рискованно, поскольку предполгается, что подобное изменение может быть чересчур непривычным для пользователей. Коллеги договорились принять решение о смене шрифтов на основе А/A/B - теста.

**Цель проекта:**

* проанализировать пользовательское поведение в приложении и принять решение о необходимости смене шрифтов 


**Задачи:**

* построить воронку событий;
* провести анализ воронки событий;
* определить на каком шаге теряется большинство пользователей;
* проверить наличие разницы между контрольными группами;
* проверить наличие разницы между контрольными и эксперементальной группами;
* принять решение о целесообразности замены шрифтов.

**Описание данных**

Каждая запись в логе — это действие пользователя, или событие.

* ***EventName*** — название события;
* ***DeviceIDHash*** — уникальный идентификатор пользователя;
* ***EventTimestamp*** — время события;
* ***ExpId*** — номер эксперимента: 246 и 247 — контрольные группы, а 248 — экспериментальная.

## Cодержание <a id="0"></a> 

* [1. Изучение и предобработка данных](#1.)
* [2. Изучение данных](#2.)
* [3.Анализ воронки событий](#3.)
* [4.Анализ результатов эксперимента](#4.)     
* [5.Общие выводы](#5.)

## Предобработка данных <a id="1."></a> 

[к содержанию](#0)

In [1]:
#импорт библиотек
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
import plotly
from plotly import graph_objects as go
import plotly.express as px
from datetime import datetime as dt
from scipy import stats as st
import math as mth
import matplotlib.ticker as ticker
warnings.filterwarnings("ignore")

In [2]:
#импорт файла
df=pd.read_csv('/datasets/logs_exp.csv', sep='\t')

FileNotFoundError: [Errno 2] File /datasets/logs_exp.csv does not exist: '/datasets/logs_exp.csv'

In [ ]:
df.head()

Для начала переименуем столбцы, для дальнейшей более удобной предобработки данных.

In [ ]:
df.columns=['event','user_id','time','group']

In [ ]:
df.info()

Пропущенные значения в данных отсутсвуют, однако формат даты не соответсувющий, его неоходимо заменить.

In [ ]:
#замена типа данных даты на соответствующий
def f(date):
    a=dt.fromtimestamp(date)
    return a

df['date_time'] = df['time'].apply(f)

In [ ]:
df.head()

Проверим дублирующие значения в переменной события.

In [ ]:
df['event'].unique()

Уникальные события не повоторяются из-за различного варианта написаний.

Проверим наличие дублирующих строк.

In [ ]:
df.duplicated().sum()

Всего в данных 413 дуюлирующих строк, навряд ли с одного и  того же устройства в один и тот же момент с точностью до секунды можно совершить одинаковое событие, скорее всего была допущена техническая ошибка при выгрузке лога. Необходимо будет выяснить причину появления дубликатов у специалиста, который занимался выгрузкой данных. На данном этапе удалим дублирующие значения, которые составляют примерно 0.17% данных

In [ ]:
df=df.drop_duplicates()

Добавим отдельный столбец даты без времени.

In [ ]:
df['date'] = df['date_time'].dt.date

In [ ]:
df['date']=pd.to_datetime(df['date'])

In [ ]:
df.head()

In [ ]:
df.info()

Теперь все типы данных являются соответсвующими.

### Вывод

В результате первичной обработки данных, мы привели к соответсвующему типа данных даты и время событий, пропущенных значений обнаружено не было. Однако проверка показала наличие дублирующих строк, их объем составлял менее 0.17% процентов от всего объема данных, логического объяснения присутствия дубликатов в контексте поставленной задачи не нашлось, поэтому данные были очищены от дублирующих строк. При этом необходимо будет вяснить причину появления дубликатов у специалиста, который занимался выгрузкой лога, возможно это был результат технической ошибки.

## Изучение данных<a id="2."></a> 

[к содержанию](#0)

* Сколько всего событий в логе?

In [ ]:
print('Всего событий в логе:', df['event'].count())

* Сколько всего пользователей в логе?

In [ ]:
print('Всего пользовталей в логе:', df['user_id'].nunique())

* Сколько в среднем событий приходится на пользователя?

In [ ]:
print((df['event'].count()/df['user_id'].nunique()).round(), 'события приходится в среднем на 1 пользователя')

* Данными за какой период вы располагаете? 

In [ ]:
df['date_time'].min()

In [ ]:
df['date_time'].max()

Данные оxватывают период с 25 июля по 7 августа 2019 года.

In [ ]:
fivethirtyeight = [ '#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52']
sns.set_palette(fivethirtyeight)
sns.set_style("darkgrid")

In [ ]:
df['date_time'].hist(figsize=(16,9), bins=25)
plt.title('Распределение событий по дате и времени', fontsize=18)
plt.xlabel('Дата и время',fontsize=13)
plt.ylabel('Количество событий', fontsize=13)
plt.xticks(rotation=30)
axs = plt.axes()
axs.xaxis.set_major_locator(ticker.MultipleLocator(1))
plt.show()

На графике распределения мы видим, что наши данные не равномерно распределены в течение всего представленного периода. До 31 июля крайне мало событий. Сложно сказать, что было не так с приложением с 25 по 31 июля, возможно в этот период при его работе возникало много багов и пользователи практически им не пользовались. В последующую неделю мы видим занчительно большее количество событий во все дни. Неделя с 25 по 31 июля кажется нетипичной для приложения, включает в себя мало событий,  а значит ее следует исключить из периода для анализа. Учитывая тот факт, что технически в логи новых дней по некоторым пользователям могут «доезжать» события из прошлого 31 июля включать в анализируемый период не будем. Определим период для анализа с 1 по 8 августа.

In [ ]:
len(df.query('date<="2019-07-31"'))/len(df)*100

Ограничивая период для анализа данных мы потеряем прмер 1,16% от всех наблюдений, что по-прежнему оставляет объем нашей выборки достаточно большим и не повлечет значительных искажений результатов.

In [ ]:
1-df.query('"2019-08-01"<=date<="2019-08-07"')['user_id'].nunique()/df['user_id'].nunique()

После ограничения периода потеряется около 0.2% пользователей, что также не является критичным.

In [ ]:
df=df.query('"2019-08-01"<=date<="2019-08-07"')

In [ ]:
df.groupby('group')['user_id'].nunique().reset_index()

После ограничений периода времени в выборке по-прежнему присутствуют примерно в равном количестве пользователи из всех экспериментальных групп.

### Вывод

В результате более подробного изучения данных мы определили:
* всего за две недели  событий в приложении было совершено 240887 событий;
* в нашей выборке 7534 уникальных пользователя;
* в среднем с каждым пользователем происходит 32 события в приложении;
* наиболее оптимальный период для проведения анализа результатов тест и воронки событий неделя с 1 по 8 агуста.

## Анализ воронки событий<a id="3."></a> 

[к содержанию](#0)

* **События в логах**

In [ ]:
events=df.groupby('event')['user_id'].count().sort_values(ascending=False).reset_index()
events.columns=['event','count']
events

In [ ]:
plt.figure(figsize=(16,9))
g=sns.barplot (y="count",x="event", data=events) 
g.set_xlabel('Событие', fontsize=13) 
g.set_ylabel('Количество', fontsize=13)
plt.xticks(fontsize=12)
plt.title('События в приложении с 1 по 8 августа 2019 года по типу', fontsize=18)
plt.show()

Всего в логах представлено пять событий, наиболее встречаемое - появление главного экрана, что ожидаемо, так как его видит каждый пользователь, открывающий приложение. Наименее встречающееся событие - просмотр тьюториала.

* **Посчитайте, сколько пользователей совершали каждое из этих событий.**

In [ ]:
us=df.groupby('user_id')['event'].nunique().reset_index()
us.columns=['id','events']
len(us.query('events==5'))

Всего 466 пользователей из лога совершали все 5 событий.

* **Отсортируйте события по числу пользователей**

In [ ]:
events_user=df.groupby('event')['user_id'].nunique().sort_values(ascending=False).reset_index()
events_user.columns=['event','users']
events_user

In [ ]:
plt.figure(figsize=(16,9))
g=sns.barplot (y="users",x="event", data=events_user) 
g.set_xlabel('Событие', fontsize=13) 
g.set_ylabel('Количество пользователей, совершивших событие', fontsize=13)
plt.xticks(fontsize=12)
plt.title('События, совершенные пользователями, в приложении с 1 по 8 августа 2019', fontsize=18)
plt.show()

Картина событий по числу пользователей аналогична с ранее представленной по числу повторений.

* **Посчитайте долю пользователей, которые хоть раз совершали событие**

In [ ]:
events_user['ratio']=events_user['users']/len(df['user_id'].unique())
events_user

98% всех пользователей открывали главную страницу приложения, 61% видели предложения из подборки, 50% пользователей заходили в офрмленную корзину, 47% пользоватлей успешно завершили покупку и только 11% всех пользователей смотрели тьюториал приложения.

* **Предположите, в каком порядке происходят события. Все ли они выстраиваются в последовательную цепочку? Их не нужно учитывать при расчёте воронки.**

Вероятнее всего цпочка событий в приложении выглядит следующим образом:
  
  - Открывая приложение, пользователь видит главный экран;
  - Затем пользователю предлагаются товары из подборки;
  - Далее пользователь переходит в корзину для дальнейшего совершения покупки;
  - Завершающим становится уведомление в приложении об успешной оплате товаров.
  
 Tutorial достаточно сложно логически включить в данную воронку из-за того, что очень малое количество пользователей совершили данное событие, несмотря на то, что по логике оно должно происходить еще до появления главной заставки приложения. Скорее всего, это объясняется тем, что как правило тьюториал показывается исключительно новым пользоватлеям для знакомства с интерфейсом приложения. Вероятно, в нашу выборку за неделю попало малое количество новых пользователей и как следствие просмотр тьюториала не являлся популярным событием. При последующем построении воронки не будем учитывать данное событие.
  

* **По воронке событий посчитайте, какая доля пользователей проходит на следующий шаг воронки (от числа пользователей на предыдущем).**

In [ ]:
funnel=events_user.drop(index=4)
#доля пользователей,которая проходит на следующий шаг воронки (от числа пользователей на предыдущем).
funnel['ratio'] =funnel['users'].div(funnel['users'].shift(1))

In [ ]:
funnel

In [ ]:
fig = go.Figure(go.Funnel(
    y = funnel['event'],
    x =funnel['users'], 
    textinfo = "value+percent initial",
    hoverinfo='x+y+percent initial+percent previous',
    opacity = 0.9,
    marker = {"color": ['#636EFA', '#EF553B', '#00CC96', '#AB63FA']},
   connector = {"line": {"color": "royalblue", "dash": "dot", "width": 2}}))
fig.update_layout(title={'text': "Воронка событий в приложении",
        'y':0.9,
        'x':0.55,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show() 

* **На каком шаге теряете больше всего пользователей?**

Больше всего пользователей теряется после просмотра главного экрана приложения - около 62% пользоватлей после главного экрана видят предложения подборки. Вероятно, оформление главной страницы приложения требует доработки, чтобы пользователи дольше задерживались на ней и уже дальше им предлагались товары.

* **Какая доля пользователей доходит от первого события до оплаты?**

48% пользователей доходит от первого события до оплаты.

### Вывод

* 98% всех пользователей открывали главную страницу приложения, 61% видели предложения из подборки, 50% пользователей заходили в офрмленную корзину, 47% пользоватлей успешно завершили покупку и только 11% всех пользователей смотрели тьюториал приложения (скорее всего это были новички, которых оказалось мало в наш анализируемый период);

* всего 466 (примерно 6% пользователей) юзеров из лога совершали все 5 событий, что говорит о том, что есть иные варианты взаимодествия с приложением;

* больше всего пользователей теряется после просмотра главного экрана приложения - только около 62% пользоватлей после главного экрана видят предложения подборки. 

* 48% пользователей доходит от первого события до оплаты.

## Анализ результатов эксперимента<a id="4."></a> 

[к содержанию](#0)

Для проверки корректности всех меанихмов и рассчетов необходимо сначала проанализирвоать результаты A/A-теста.

* **Количество пользователей в каждой экспериментальной группе**

In [ ]:
df.groupby('group')['user_id'].nunique().reset_index()

Количество пользователей в каждой группе различается не более чем на 1-2%.

* **Пересечния между группами**

In [ ]:
#создаем списки с уникальными юзерами каждой группы
gr246=df.query('group==246')['user_id'].unique().tolist()
gr247=df.query('group==247')['user_id'].unique().tolist()
gr248=df.query('group==248')['user_id'].unique().tolist()

Проверяем, не попали ли некоторые пользователи одновременно в несколько групп

In [ ]:
any(item in gr246 for item in gr247)

In [ ]:
any(item in gr247 for item in gr248)

In [ ]:
any(item in gr246 for item in gr248)

Пересечений между группами по пользователям нет.

* **Выберите самое популярное событие. Посчитайте число пользователей, совершивших это событие в каждой из контрольных групп. Посчитайте долю пользователей, совершивших это событие. Проверьте, будет ли отличие между группами статистически достоверным. Проделайте то же самое для всех других событий (удобно обернуть проверку в отдельную функцию). Можно ли сказать, что разбиение на группы работает корректно?**

Проведем z-тест чтобы проверить наличие статистически значимых различий между группами
* *H0*: доли в разных группах равны
* *H1*: доли в разных группах различаются

In [ ]:
df3=df.query('event!="Tutorial"')

In [ ]:
df4=df3.pivot_table(index='event',columns='group', values='user_id', aggfunc='nunique').sort_values(by=246,ascending=False)
df4['sum']=df4[246]+df4[247]

In [ ]:
df4

In [ ]:
users=df.groupby('group')['user_id'].nunique()
users = users.to_frame().reset_index()
users.loc[3] = ['sum', 4997]
users = users.set_index(users.columns[0])
users

In [ ]:
def test(g1,g2,event,alpha):
    
    p1=df4.loc[event, g1]/users.loc[g1, 'user_id']
    p2=df4.loc[event, g2]/users.loc[g2, 'user_id']
    p_comb=(df4.loc[event, g1]+df4.loc[event, g2])/(users.loc[g1, 'user_id']+users.loc[g2, 'user_id'])
    diff=p1-p2
    
    z=diff/mth.sqrt(p_comb * (1 - p_comb) * (1 /users.loc[g1, 'user_id'] + 1 / users.loc[g2, 'user_id']))
    
    distr = st.norm(0, 1)
    
    p_value = (1 - distr.cdf(abs(z))) * 2
    
    print('Группы:  {} и {}\nCобытие: {}\np-value: {p_value:.3f}'.format(g1, g2, event, p_value=p_value))
    if (p_value < alpha):
        print("Отвергаем нулевую гипотезу: между долями есть значимая разница")
    else:
        print("Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными")

In [ ]:
for event in df4.index:
    test(246, 247, event, 0.05)
    print()

### Вывод

Разница в контрольных группах не выявлена. АА-тест удовлетворяет всем критериям успешности:

- Количество пользователей в различных группах различается не более, чем на 1-2%;
- перечение групп по пользователям отсутствует;
- для всех групп фиксируются одни и те же события;
- различие ключевой метрики по группам не превышает 1% и не имеет статистической значимости.
  

* **Аналогично поступите с группой с изменённым шрифтом. Сравните результаты с каждой из контрольных групп в отдельности по каждому событию. Сравните результаты с объединённой контрольной группой. Какие выводы из эксперимента можно сделать?**

In [ ]:
for event in df4.index:
    test(246, 248, event, 0.05)
    print()

In [ ]:
for event in df4.index:
    test(247, 248, event, 0.05)
    print()

In [ ]:
for event in df4.index:
    test('sum', 248, event, 0.05)
    print()

### Вывод

На пятипроцентном уровне значимости во всех тестах у нас не было оснований отвергнуть нулевые гипотезы о равентсве долей между группами пользователей. Что говорит нам о том, что изменение шрифтов в приложении не повлияет на пользовательское поведение.

* **Уровень значимости при проверке статистических гипотез**

Всего было проведено 4 АА-теста и 12 АB-тестов. При проведении тестов использовался пятипроцентный уровень значимости. Учитывая что нами было проведено достаточно большое количество тестов на трех выборках из одного и того же набора данных, у нас увеличилась вероятность ошибки первого рода. В связи с этим можно воспользоваться поправкой Бонферони и скорректировать уровень значимости с учетом количества групп (A1, A2, A, B - 4) И количества тестируемых событий воронки (4), то есть задать alpha=0.05/16. Проводить повторно тесты с учетом корретировки уровня значимости (альфы) в меньшую сторону нет необходимости, поскольку при пятипроцентном уровне значимости не было оснваний отвергнуть нулевые гипотезы.

## Общие выводы<a id="5."></a> 

[к содержанию](#0)

В результате проведенного анализа воронки событий и А/А/В-теста можно сделать следующие выводы:

* Только примерно 6% пользователей из лога совершали все 5 событий, что говорит о том, что есть иные варианты взаимодействия с приложением отличающиеся от "тьюториала после установки-главного экрана-предложения подборки-корзина-покупка";


* конверсия в успешные покупки составляет примерно 48%;


* больше всего пользователей теряется после просмотра главного экрана приложения - около 62% пользоватлей после главного экрана видят предложения подборки. Вероятно, оформление главной страницы приложения требует доработки с целью более длительного удержания на ней пользоватлей и затем предложения им товаров из подборки. Далее по воронке событий не наблюдается значительной потери клинетов;


* результаты А/А/В-теста говорят о том, что смена шрифтов в приложении не повлияет на поведение пользователей, поэтому реализовывать подобную идею не целесообразно, поскольку разработка новых шрифтов потребует определенных затрат, а пользователи будут приносить все ту же прибыль. 



